In [1]:
import pygame as pg
import sys
import random as rd

pygame 2.1.2 (SDL 2.0.16, Python 3.10.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


class Screen:
    def __init__(self, title, wh):
        pg.display.set_caption(title)
        self.screen_sfc = pg.display.set_mode(wh)  # Surface
        self.screen_rct = self.screen_sfc.get_rect()  # Rect

        self.width = self.screen_rct.width
        self.height = self.screen_rct.height



# 迷路を作る
## Screen 関数
- window を作成する
## maze 関数
- 迷路を作成する
  - タイルを配置する
  - サンプルでは真四角の画像を用いている
    - pygameに四角を描画するものはあるのか
      - draw_rect
      - 左上と右下の座標を使う
        - タイルの幅を規定する。
          - tile_length
    - rowとcolumnで二重for文を使ってmazeの二次元リストを作成している
      - 画像とサイズの情報を用いて描画


In [2]:
class Screen:

    def __init__(self) -> None:
        
        pg.display.set_caption("こうかとん迷路")

        self.x = 16 # 迷路のxマス数
        self.y = 9 # 迷路のyマス数
        self.tile_length = 100 #タイルの大きさ

        self.sfc = pg.display.set_mode((self.x*self.tile_length+14,self.y*self.tile_length+17))
        self.rct = self.sfc.get_rect()

        self.wall_color = (0,100,0)
        self.floor_color = (216,208,104)

        """ self.maze_map = [[rd.randint(0,1) for j in range(self.x)] for i in range(self.y)] """
        self.maze_map = self.make_maze(self.x,self.y)

    def draw_map(self):
        self.sfc.fill(self.floor_color)
        for i in range(self.y):
            for j in range(self.x):
                if self.maze_map[i][j] == 1:
                    pg.draw.rect(
                        self.sfc,
                        color=self.wall_color,
                        rect=(j*(self.tile_length+1),i*(self.tile_length+1),(self.tile_length-1),(self.tile_length-1)),
                        width=0)

    def make_maze(self,yoko, tate):
        XP = [ 0, 1, 0, -1]
        YP = [-1, 0, 1,  0]

        maze_lst = []
        for y in range(tate):
            maze_lst.append([0]*yoko)
        for x in range(yoko):
            maze_lst[0][x] = 1
            maze_lst[tate-1][x] = 1
        for y in range(1, tate-1):
            maze_lst[y][0] = 1
            maze_lst[y][yoko-1] = 1
        for y in range(2, tate-2, 2):
            for x in range(2, yoko-2, 2):
                maze_lst[y][x] = 1
        for y in range(2, tate-2, 2):
            for x in range(2, yoko-2, 2):
                if x > 2: rnd = rd.randint(0, 2)
                else:     rnd = rd.randint(0, 3)
                maze_lst[y+YP[rnd]][x+XP[rnd]] = 1

        return maze_lst

In [3]:
def make_maze(yoko, tate):
    XP = [ 0, 1, 0, -1]
    YP = [-1, 0, 1,  0]

    maze_lst = []
    for y in range(tate):
        maze_lst.append([0]*yoko)
    for x in range(yoko):
        maze_lst[0][x] = 1
        maze_lst[tate-1][x] = 1
    for y in range(1, tate-1):
        maze_lst[y][0] = 1
        maze_lst[y][yoko-1] = 1
    for y in range(2, tate-2, 2):
        for x in range(2, yoko-2, 2):
            maze_lst[y][x] = 1
    for y in range(2, tate-2, 2):
        for x in range(2, yoko-2, 2):
            if x > 2: rnd = rd.randint(0, 2)
            else:     rnd = rd.randint(0, 3)
            maze_lst[y+YP[rnd]][x+XP[rnd]] = 1

    return maze_lst

def show_maze(canvas, maze_lst):
    color = ["white", "gray"]
    for y in range(len(maze_lst)):
        for x in range(len(maze_lst[y])):
            canvas.create_rectangle(x*100, y*100, x*100+100, y*100+100, 
                                    fill=color[maze_lst[y][x]])

In [10]:
class Bird:
    def __init__(self, image, size, xy, scr):
        self.sfc = pg.image.load(image)
        self.sfc = pg.transform.rotozoom(self.sfc, 0, size)
        self.rct = self.sfc.get_rect()
        self.rct.center = xy
        self.size = size

        self.xy = 1+1j
        self.tile_size = scr.tile_length

    """ def check_bound(self,rct, scr_rct):
        yoko, tate = +1, +1  # 領域内
        if rct.left < scr_rct.left or scr_rct.right < rct.right:
            yoko = -1  # 領域外
        if rct.top < scr_rct.top or scr_rct.bottom < rct.bottom:
            tate = -1  # 領域外
        return yoko, tate """

    def update(self, scr):

        tmp = self.xy
        key_states = pg.key.get_pressed()  # 辞書
        if key_states[pg.K_UP]:
            """ self.rct.centery -= 1 """
            self.xy -= 1j
        if key_states[pg.K_DOWN]:
            """ self.rct.centery += 1 """
            self.xy += 1j
        if key_states[pg.K_LEFT]:
            """ self.rct.centerx -= 1 """
            self.xy -= 1
        if key_states[pg.K_RIGHT]:
            """ self.rct.centerx += 1 """
            self.xy += 1

        if scr.maze_map[int(self.xy.imag)][int(self.xy.real)] != 1:
            self.rct.center = (int(self.xy.real)*self.tile_size+(self.rct.size[0]//2),int(self.xy.imag)*self.tile_size+(self.rct.size[1]//2))
        else:
            self.xy = tmp
        # 練習7
        """ if self.check_bound(self.rct, scr.rct) != (1, 1):  # 領域外だったら
            if key_states[pg.K_UP]:
                self.rct.centery += 1
            if key_states[pg.K_DOWN]:
                self.rct.centery -= 1
            if key_states[pg.K_LEFT]:
                self.rct.centerx += 1
            if key_states[pg.K_RIGHT]:
                self.rct.centerx -= 1 """
        scr.sfc.blit(self.sfc,self.rct)

In [5]:
print(make_maze(16,9))

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1], [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1], [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [17]:
def main():
    running = True
    scr = Screen()
    kkt = Bird("fig/6.png", 2.0, (900, 400),scr)
    clock = pg.time.Clock()


    while running:
        # ここにコードを書く

        scr.draw_map()
        kkt.update(scr)

        print(kkt.xy)
        
        # イベント処理部
        for event in pg.event.get():
            if event.type == pg.QUIT:  # 終了イベント
                running = False
                pg.quit()  #pygameのウィンドウを閉じる
                sys.exit() #システム終了            

        pg.display.update() #描画処理を実行
        clock.tick(20)

In [18]:
main()

(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(1+1j)
(2+1j)
(3+1j)
(4+1j)
(4+1j)
(4+1j)
(4+1j)
(5+1j)
(5+1j)
(5+1j)
(5+1j)
(4+1j)
(3+1j)
(2+1j)
(2+1j)
(2+1j)
(2+1j)
(2+1j)
(2+1j)
(2+1j)
(2+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(4+1j)
(5+1j)
(5+1j)
(5+1j)
(5+1j)
(5+1j)
(5+1j)
(6+1j)
(6+1j)
(6+1j)
(6+1j)
(7+1j)
(8+1j)
(9+1j)
(9+1j)
(9+1j)
(9+1j)
(9+1j)
(9+1j)
(9+1j)
(9+1j)
(9+1j)
(8+1j)
(7+1j)
(7+1j)
(7+1j)
(7+1j)
(6+1j)
(5+1j)
(5+1j)
(5+1j)
(5+1j)
(5+1j)
(4+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+1j)
(3+2j)
(3+3j)
(3+3j)
(3+3j)
(3+3j)
(3+3j)
(3+3j)
(3+3j)
(3+3j)
(2+3j)
(2+3j)
(2+3j)
(2+3j)
(2+3j)
(2+3j)
(2+3j)
(2+3j)
(2+3j)
(1+3j)
(1+3j)
(1+3j)
(1+3j)
(1+3j)
(1+3j)
(1+4j)
(1+5j)
(1+5j)
(1+5j)
(1+6j)
(1+7j)
(1+7j)
(1+7j)
(1+7j)
(1+7j)
(1+7j)
(1+7j)
(1+7j)
(2+7j)
(3+7j)
(3+7j)
(3+7j)
(3+7j)
(4+7j)
(5+7j)
(5+7j)
(5+7j)
(5+7j)
(5+7j)
(5+7j)
(5+7j)
(5+6j)
(5+5j)
(5+5j)
(5+5j)
(5+5j)
(5+5j)
(5+5j)
(5+5j)
(5+6j)
(5+7j)

SystemExit: 